In [ ]:

'''
@article{oviedoBankNote-Net2022,
  title   = {BankNote-Net: Open Dataset for Assistive Currency Recognition},
  author  = {Felipe Oviedo, Srinivas Vinnakota, Eugene Seleznev, Hemant Malhotra, Saqib Shaikh & Juan Lavista Ferres},
  journal = {https://arxiv.org/pdf/2204.03738.pdf},
  year    = {2022},
}
'''


#TRAIN FROM EMBEDDING

#training model


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [3]:
!unzip "/content/gdrive/MyDrive/banknote-net.zip"

Archive:  /content/gdrive/MyDrive/banknote-net.zip
   creating: banknote-net/
  inflating: banknote-net/CODE_OF_CONDUCT.md  
  inflating: banknote-net/.DS_Store  
  inflating: __MACOSX/banknote-net/._.DS_Store  
  inflating: banknote-net/LICENSE    
  inflating: banknote-net/env.yaml   
   creating: banknote-net/models/
  inflating: banknote-net/README.md  
  inflating: banknote-net/reload     
  inflating: banknote-net/.gitignore  
   creating: banknote-net/.git/
   creating: banknote-net/data/
  inflating: banknote-net/SECURITY.md  
   creating: banknote-net/src/
  inflating: banknote-net/models/banknote_net_encoder.h5  
  inflating: banknote-net/.git/config  
   creating: banknote-net/.git/objects/
  inflating: banknote-net/.git/HEAD  
   creating: banknote-net/.git/info/
   creating: banknote-net/.git/logs/
  inflating: banknote-net/.git/description  
   creating: banknote-net/.git/hooks/
   creating: banknote-net/.git/refs/
  inflating: banknote-net/.git/index  
  inflating: bankn

In [4]:
import pandas as pd
PATH = "/content/banknote-net/data/banknote_net.feather"
data = pd.read_feather(PATH)
len(data.Currency.unique())


17

In [5]:
#extract class names
#https://www.tensorflow.org/tutorials/images/classification#predict_on_new_data

features,label = data.iloc[:,:-2],data.iloc[:,-2:]
combine_label = data['Currency'].astype(str) + data['Denomination'].astype(str)
#combine_label = combine_label.unique()
combine_label

0        AUD100_1
1        AUD100_1
2        AUD100_1
3        AUD100_1
4        AUD100_1
           ...   
24821      USD5_2
24822      USD5_2
24823      USD5_2
24824      USD5_2
24825      USD5_2
Length: 24826, dtype: object

In [6]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
def create_generator(
    TRAIN_PATH: str,
    VAL_PATH:str,
    IMG_SIZE: tuple,
    BATCH_SIZE: int = 2,
    NUM_CLASSES: int = 10,
):
  IMG_WIDTH,IMG_HEIGHT = IMG_SIZE
  train_datagen = ImageDataGenerator(rescale=1.0/255,
                                     samplewise_center=False,
                                     samplewise_std_normalization=False,
                                     rotation_range=180,
                                     channel_shift_range=40,
                                     fill_mode='nearest',
                                     )
  test_datagen = ImageDataGenerator(rescale=1.0/255,)

  #initialize generators and create TF datasets
  train_generator = train_datagen.flow_from_directory(TRAIN_PATH,
                                                      target_size = (IMG_WIDTH,IMG_HEIGHT),
                                                      batch_size = BATCH_SIZE,
                                                      shuffle=True,
                                                      seed=12345,
                                                      class_mode = "categorical",
                                                      )
  validation_generator  = test_datagen.flow_from_directory(
      VAL_PATH,
      target_size = (IMG_WIDTH,IMG_HEIGHT),
      batch_size = BATCH_SIZE,
      shuffle = False,
      class_mode = "categorical",
  )
  train_ds = tf.data.Dataset.from_generator(
      lambda:train_generator,
      output_types = (tf.float32,tf.float32),
      output_shapes = ([None,IMG_HEIGHT,IMG_WIDTH, 3],[None,NUM_CLASSES]),
  )
  val_ds = tf.data.Dataset.from_generator(
      lambda:validation_generator,
      output_types=(tf.float32,tf.float32),
      output_shapes=([None,IMG_HEIGHT,IMG_WIDTH,3],[None,NUM_CLASSES]),
  )
  return train_ds,val_ds
  


In [7]:
import os
import numpy as np
BATCH_SIZE = 4
NB_EPOCH = 25
ENC_PATH = "/content/banknote-net/models/banknote_net_encoder.h5"
DATA_PATH = "/content/banknote-net/data/example_images/SEK"
NUM_CLASSES = len(next(os.walk(f"{DATA_PATH}/train/"))[1])
IMG_SIZE = (224, 224)
NB_TRAINING_SAMPLES= sum([len(file) for r,d, file in os.walk(f"{DATA_PATH}/train/")])
NB_VALIDATION_SAMPLES = sum([len(files) for r, d, files in os.walk(f"{DATA_PATH}/val/")])

train_ds,val_ds = create_generator(TRAIN_PATH=f"{DATA_PATH}/train/",
                                   VAL_PATH=f"{DATA_PATH}/val/",
                                   IMG_SIZE=IMG_SIZE,
                                   BATCH_SIZE=BATCH_SIZE,
                                   NUM_CLASSES=NUM_CLASSES,)
'''
encoder = load_model(ENC_PATH)
for layer in encoder.layers:
  layer.trainable = False

input = Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = encoder(input)
x = Dense(256,activation='relu')(input)
x = Dropout(0.5)(x)
x = Dense(NUM_CLASSES, activation='softmax')(x)
model = Model(inputs=input,outputs=x)
'''
model1 = load_model(ENC_PATH)
model1.summary()

'''
checkpoint = ModelCheckpoint(
      filepath = ("/content/banknote-net/src/trained_models/custom_classifier.h5"),
      monitor = 'val_acc',
      save_best_only=True,
)

  #compile and fit
model.compile(loss='categorical_crossentropy',
                optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3),
                metrics=['acc',
                         tf.keras.metrics.Precision(),
                         tf.keras.metrics.Recall(),
                         ],
                )
model.fit(train_ds,
            steps_per_epoch = NB_TRAINING_SAMPLES//BATCH_SIZE,
            epochs = NB_EPOCH,
            validation_steps=NB_VALIDATION_SAMPLES // BATCH_SIZE + 1,
            validation_data= val_ds,
            callbacks = [checkpoint],
            )
'''

Found 41 images belonging to 8 classes.
Found 15 images belonging to 8 classes.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.30_224 (Funct  (None, 7, 7, 1664)       3766048   
 ional)                                                          
                                                                 
 global_average_pooling2d (G  (None, 1664)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 256)               426240    
                                                                 
Total params: 4,192,288
Trainable params: 3,733,056
Non-trainable params: 459,232
_________________________________________________________________


'\ncheckpoint = ModelCheckpoint(\n      filepath = ("/content/banknote-net/src/trained_models/custom_classifier.h5"),\n      monitor = \'val_acc\',\n      save_best_only=True,\n)\n\n  #compile and fit\nmodel.compile(loss=\'categorical_crossentropy\',\n                optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3),\n                metrics=[\'acc\',\n                         tf.keras.metrics.Precision(),\n                         tf.keras.metrics.Recall(),\n                         ],\n                )\nmodel.fit(train_ds,\n            steps_per_epoch = NB_TRAINING_SAMPLES//BATCH_SIZE,\n            epochs = NB_EPOCH,\n            validation_steps=NB_VALIDATION_SAMPLES // BATCH_SIZE + 1,\n            validation_data= val_ds,\n            callbacks = [checkpoint],\n            )\n'

In [8]:
BATCH_SIZE = 128
NB_EPOCH = 25
PATH = "/content/banknote-net/data/banknote_net.feather"

  #load data from embedding
data = pd.read_feather(PATH)
data.Currency.unique()
  #data = data[data.Currency == CURRENCY]
  
data= data.sample(frac = 1)
#labels = data[["Currency","Denomination"]].copy()
labels = data['Currency'] + data['Denomination']
labels = labels.astype("category")
labels_encoded = pd.get_dummies(labels)
data = data.iloc[:,:-2] #keep the embedding

In [9]:
#model II for the compare result
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.models import Model, load_model
def main(data, labels, labels_encoded):
  "trains shallow classifier using embeddings"
  CURRENCY = [
    'AUD', #australian dollar
    'BRL', #Brazillian real
    'CAD', #Canadian dollar
    'EUR', #Euro
    'GBP', #Pound
    'INR', #Indian Rupee
    'JPY', #Japanese Yen
    'MXN', # Mexican Peso
    'PKR', #Pakistani peso
    'SGD', #Singapore dollar
    'TRY', #Turkish lira
    'USD', #US dollar
    'NZD', #newzealand dollar
    'NNR', #
    'MYR', #Malaysian Ringgit
    'IDR', #Indonesian rupiah
    'PHP', #philipine peso
  ]

  #define dataset and shallow model
  NUM_CLASSES  = len(labels.unique())
  NB_TRAINING_SAMPLES = len(data)

  input = Input(shape=(256,))
  ##### test load pretrained model
  #encoder = load_model("/content/banknote-net/models/banknote_net_encoder.h5")
  #for layer in model1.layers:
  #      layer.trainable = False
  #x = model1(input)
  x = Dense(256,activation='relu')(input)
  x = Dropout(0.5)(x)
  x = Dense(NUM_CLASSES, activation='softmax')(x)
  model = Model(inputs=input,outputs=x)
  model.summary()
  print("model build: ok")
  checkpoint = ModelCheckpoint(
      filepath = ("/content/banknote-net/models/banknote_net_encoder.h5"),
      monitor = 'val_acc',
      save_best_only=True,
  )

  #compile and fit
  model.compile(loss='categorical_crossentropy',
                optimizer = tf.keras.optimizers.Adam(lr=1e-3),
                metrics=['acc',
                         tf.keras.metrics.Precision(),
                         tf.keras.metrics.Recall(),
                         ],
                )
  model.fit(x=data.values,
            y = labels_encoded.values,
            steps_per_epoch = NB_TRAINING_SAMPLES//BATCH_SIZE,
            epochs = NB_EPOCH,
            validation_split = 0.2,
            callbacks = [checkpoint],
            )
  
  return model

In [10]:
model2 = main(data,labels,labels_encoded)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256)]             0         
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 224)               57568     
                                                                 
Total params: 123,360
Trainable params: 123,360
Non-trainable params: 0
_________________________________________________________________


model build: ok
Epoch 1/25
193/193 [==============================] - 2s 8ms/step - loss: 2.1519 - acc: 0.5803 - precision: 0.9463 - recall: 0.4275 - val_loss: 0.4785 - val_acc: 0.9001 - val_precision: 0.9762 - val_recall: 0.8268
Epoch 2/25
193/193 [==============================] - 1s 6ms/step - loss: 0.6072 - acc: 0.8525 - precision: 0.9463 - recall: 0.7838 - val_loss: 0.3467 - val_acc: 0.9170 - val_precision: 0.9710 - val_recall: 0.8709
Epoch 3/25
193/193 [==============================] - 1s 6ms/step - loss: 0.4478 - acc: 0.8854 - precision: 0.9547 - recall: 0.8377 - val_loss: 0.2851 - val_acc: 0.9283 - val_precision: 0.9695 - val_recall: 0.8973
Epoch 4/25
193/193 [==============================] - 1s 6ms/step - loss: 0.3788 - acc: 0.8991 - precision: 0.9553 - recall: 0.8626 - val_loss: 0.2683 - val_acc: 0.9305 - val_precision: 0.9684 - val_recall: 0.9068
Epoch 5/25
193/193 [==============================] - 2s 8ms/step - loss: 0.3257 - acc: 0.9121 - precision: 0.9573 - recall: 0.8

In [11]:
five_usd = "/content/sample_data/one_uds.jpg"
img = tf.keras.utils.load_img(
    five_usd, target_size=(224, 224)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model1.predict(img_array)


1/1 [==============================] - 2s 2s/step


In [ ]:
predictions.shape

(1, 256)

In [ ]:
result1 = np.argmax(predictions,axis=1)

In [ ]:
print(result1)

[44]


In [12]:
result = model2.predict(predictions)
result.shape

1/1 [==============================] - 0s 45ms/step


(1, 224)

In [15]:
result = np.argmax(result,axis = 1)

In [19]:
checker = labels.unique()
print(checker[result])

['NZD100_2']
Categories (224, object): ['AUD100_1', 'AUD100_2', 'AUD10_1', 'AUD10_2', ..., 'USD50_1', 'USD50_2',
                           'USD5_1', 'USD5_2']


In [ ]:
denomination_list = combine_label.unique()

In [ ]:

deno_trim_list= []
for i in denomination_list:
  k = ''
  for j in i:
    if j != "_":
      k += j
    else:
      if k not in deno_trim_list:
        deno_trim_list.append(k)
      break

In [ ]:
combine_label.unique()

array(['AUD100_1', 'AUD100_2', 'AUD10_1', 'AUD10_2', 'AUD20_1', 'AUD20_2',
       'AUD50_1', 'AUD50_2', 'AUD5_1', 'AUD5_2', 'BRL100_1', 'BRL100_2',
       'BRL10_1', 'BRL10_2', 'BRL200_1', 'BRL200_2', 'BRL20_1', 'BRL20_2',
       'BRL2_1', 'BRL2_2', 'BRL50_1', 'BRL50_2', 'BRL5_1', 'BRL5_2',
       'CAD100_1', 'CAD100_2', 'CAD10_1', 'CAD10_2', 'CAD20_1', 'CAD20_2',
       'CAD50_1', 'CAD50_2', 'CAD5_1', 'CAD5_2', 'EUR100_1', 'EUR100_2',
       'EUR10_1', 'EUR10_2', 'EUR200_1', 'EUR200_2', 'EUR20_1', 'EUR20_2',
       'EUR50_1', 'EUR50_2', 'EUR5_1', 'EUR5_2', 'GBP10_1_1', 'GBP10_1_2',
       'GBP10_2_1', 'GBP10_2_2', 'GBP20_1_1', 'GBP20_1_2', 'GBP20_2_1',
       'GBP20_2_2', 'GBP50_1', 'GBP50_2', 'GBP5_1', 'GBP5_2',
       'IDR100000_1', 'IDR100000_2', 'IDR1000_1', 'IDR1000_2',
       'IDR20000_1', 'IDR20000_2', 'IDR2000_1', 'IDR2000_2', 'IDR50000_1',
       'IDR50000_2', 'IDR5000_1', 'IDR5000_2', 'INR1000_1', 'INR1000_2',
       'INR100_1_1', 'INR100_1_2', 'INR100_2_1', 'INR100_2_2',
  